## Import lib

In [1]:
#!!!
import os
import pandas as pd
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix,accuracy_score,classification_report
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.ensemble import EasyEnsembleClassifier


# from sklearn.datasets import load_files
# sys.path.append('..') 
# import helpers.data_mining_helpers as dmh

# import nltk
# nltk.download('punkt')
# import seaborn as sns
# from sklearn.decomposition import PCA
# from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
# from sklearn.preprocessing import binarize
# from sklearn.metrics.pairwise import cosine_similarity



## Data input

In [2]:
#!!!
train_file_path = '..\\training.csv'
test_file_path = '..\\public_processed.csv'
test_file_answer_path = '..\\public.csv'
test_private_file_path = '..\\private_1_processed.csv'

# train_file_path = 'training.csv'
# test_file_path = 'public_processed.csv'
# test_file_answer_path = 'public.csv'
# test_private_file_path = 'private_1_processed.csv'

df = pd.read_csv(train_file_path)
testing_df = pd.read_csv(test_file_path)
testing_answer_df = pd.read_csv(test_file_answer_path)
testing_private_df = pd.read_csv(test_private_file_path)

df.shape : (8688526, 26)  |||  test_df.shape : (600182, 25) ||| testing_answer_df.shape : (609040, 26) ||| testing_private_df.shape : (754139, 25)

In [3]:
#!!!
training_df = df.copy()
test_df = testing_df.copy()
# y = pd.concat([training_df['label'],testing_answer_df['label']],axis=0) 
y = training_df['label']
training_df = training_df.drop(['label','txkey'], axis=1)
test_df = test_df.drop(['txkey'], axis=1)
# testing_answer_df = testing_answer_df.drop(['txkey'], axis=1)
pridict = pd.DataFrame(testing_private_df['txkey'] )
testing_private_df = testing_private_df.drop(['txkey'], axis=1)
test_answer_df = testing_answer_df.copy()
test_answer_df = test_answer_df.drop(['label','txkey'], axis=1)
# training_df = pd.concat([training_df,test_answer_df],axis=0)

In [4]:
#!!!
train_num = training_df.shape[0]
test_num = test_df.shape[0]
private_num = test_num + train_num
allData = pd.concat([training_df ,test_df, testing_private_df], axis=0)


In [5]:
#!!!
label_encoder_chid = LabelEncoder()
label_encoder_cano = LabelEncoder()
label_encoder_mchno = LabelEncoder()
label_encoder_acqic = LabelEncoder()
label_encoder_etymd = LabelEncoder()
label_encoder_mcc = LabelEncoder()
label_encoder_stocn = LabelEncoder()
label_encoder_scity = LabelEncoder()
label_encoder_stscd = LabelEncoder()
label_encoder_hcefg = LabelEncoder()
label_encoder_csmcu = LabelEncoder()

allData['chid'] = label_encoder_chid.fit_transform(allData['chid'])
allData['cano'] = label_encoder_cano.fit_transform(allData['cano'])
allData['mchno'] = label_encoder_mchno.fit_transform(allData['mchno'])
allData['acqic'] = label_encoder_acqic.fit_transform(allData['acqic'])
allData['etymd']= label_encoder_etymd.fit_transform(allData['etymd'])
allData['mcc']= label_encoder_mcc.fit_transform(allData['mcc'])
allData['stocn']= label_encoder_stocn.fit_transform(allData['stocn'])
allData['scity']= label_encoder_scity.fit_transform(allData['scity'])
allData['stscd']= label_encoder_stscd.fit_transform(allData['stscd'])
allData['hcefg']= label_encoder_hcefg.fit_transform(allData['hcefg'])
allData['csmcu']= label_encoder_csmcu.fit_transform(allData['csmcu'])


allData['chid'].astype("category")
allData['cano'].astype("category")
allData['mchno'].astype("category")
allData['acqic'].astype("category")
allData['etymd'].astype("category")
allData['mcc'].astype("category")
allData['stocn'].astype("category")
allData['scity'].astype("category")
allData['stscd'].astype("category")
allData['hcefg'].astype("category")
allData['csmcu'].astype("category")

allData['contp'].astype("category")
allData['etymd'].astype("category")
allData['ecfg'].astype("category")
allData['insfg'].astype("category")
allData['bnsfg'].astype("category")
allData['ovrlt'].astype("category")
allData['flbmk'].astype("category")
allData['flg_3dsmk'].astype("category")



0         0
1         0
2         0
3         0
4         0
         ..
754134    0
754135    0
754136    0
754137    1
754138    0
Name: flg_3dsmk, Length: 10042847, dtype: category
Categories (2, int64): [0, 1]

In [6]:
#!!!
training_df = allData[:train_num]
test_df = allData[train_num:private_num]
testing_private_df = allData[private_num: ]


In [ ]:
print(train_num, test_num, private_num)

## Data preprocess.

type:  
type0 : txkeys     
type1 (times) : locdt loctm        
type2 (money) : conam flam1 ovrlt csmcu csmam      
type3 (person) : chid cano     
type4 (places) : mchno stocn scity     
type5 (other trade types) : contp etymd acqic mcc ecfg insfg iterm bnsfg stscd flbmk hcefg flg_3dsmk       


analysis part.

In [7]:
#!!!




fraud_df = training_df[y == 1]
non_fraud_df = training_df[y == 0]
# print(fraud_df.shape)
# print(non_fraud_df.shape)
# print(training_df.shape)
display(fraud_df)
display(non_fraud_df)


,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
318,0,190457,421554,285835,5,5,96567,450,455,5741.00,...,5752,46,3872,5,0,0,6,68,5752,0
339,49,204814,336900,274723,5,7,47446,6696,300,33.00,...,33,120,7624,1,0,0,6,68,33,0
881,19,43049,93500,2543,5,7,121541,4266,252,0.00,...,0,120,5893,1,0,0,6,66,0,0
1576,47,213335,154685,620474,5,5,97869,7688,366,1202.83,...,1201,27,13057,5,0,0,6,77,38,0
1598,14,213413,47112,52430,5,5,137008,6406,300,1790.00,...,1797,58,12019,1,0,0,6,68,1797,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8686800,48,33914,462522,65729,5,7,129765,1199,300,50.00,...,50,120,9796,1,0,0,6,68,50,0
8686969,26,100301,144233,49864,4,5,101491,1744,328,774.00,...,773,21,7974,5,0,0,6,68,773,0
8687406,14,95753,142031,173302,4,1,37828,4590,329,0.00,...,0,120,11787,5,0,0,6,66,0,0
8687798,44,63707,310891,53057,5,1,47446,4456,300,490.00,...,491,120,7624,5,0,0,6,68,491,0


,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
0,46,101812,256109,154065,5,1,138119,4871,351,1524.0,...,1529,0,11874,5,0,0,6,68,1529,0
1,17,85509,297980,350989,5,7,45877,1881,258,101.0,...,101,0,11874,5,0,0,8,68,101,0
2,44,212641,425640,76572,5,4,62433,4871,264,117.0,...,116,0,11874,5,0,0,6,68,116,0
3,42,102702,168531,167846,5,1,138119,1844,351,1676.0,...,1683,0,11874,5,0,0,6,68,1683,0
4,31,185737,493852,407593,5,5,55077,1881,382,14.0,...,14,0,11874,5,0,0,6,68,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688521,14,84737,325181,174672,5,4,41109,3193,258,49.0,...,49,0,11874,5,0,0,0,68,49,0
8688522,23,105923,439282,104273,5,4,167076,4871,252,4020.0,...,4018,0,11874,5,0,0,6,68,4018,0
8688523,23,141030,257694,103443,5,4,65068,1613,248,315.0,...,314,0,8472,5,0,0,6,68,314,0
8688524,10,120209,57843,630419,5,5,136008,1881,248,226.0,...,225,0,11874,5,0,0,6,68,225,0


In [ ]:
# each feature should use ifts own label encoder so that the unseen value can be handled
# try:
#     print(label_encoder.transform([-1]))
# except ValueError:
#     print("unseen value")


In [ ]:
# training_df.drop(['txkey'], axis=1, inplace=True)

Create two additional feature:   
1. prob_ratio (100-0)
2. prob_ratio_101 (1, 0, -1)


In [8]:
#!!!
prob_ratio_df = training_df.loc[:,['cano']].copy()
prob_ratio_101_df = training_df.loc[:,['cano']].copy()
test_prob_ratio_df = test_df.loc[:,['cano']].copy()
test_prob_ratio_101_df = test_df.loc[:,['cano']].copy()
private_prob_ratio_df = testing_private_df.loc[:,['cano']].copy()
private_prob_ratio_101_df = testing_private_df.loc[:,['cano']].copy()

In [ ]:
# print(fraud_df[col].shape[0]/ training_df[col].shape[0])
# print(non_fraud_df[col].shape[0]/ training_df[col].shape[0])
for col in training_df.columns:  
    print(f"col: {col}")
    # print(f"nunique: {training_df[col].nunique()}")
    # print(f"unique values: {training_df[col].unique()}")
    # print(f"fraud nunique: {fraud_df[col].nunique()}")
    # print(f"fraud unique values: {fraud_df[col].unique()}")
    # print(f"non fraud nunique: {non_fraud_df[col].nunique()}")
    # print(f"non fraud unique values: {non_fraud_df[col].unique()}")
    # print(fraud_df[col].nunique()/ training_df[col].nunique())
    # print(non_fraud_df[col].nunique()/ training_df[col].nunique()) 
    # unique_values = fraud_df[col].unique()
    # print(f"unique values: {unique_values}")

    #average_ratio
    # average_ratio = 0
    # for value in unique_values:
    #     average_ratio += fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
    # average_ratio /= len(unique_values)
    # print(f"average_ratio: {average_ratio}")

    #real_ratio
    # unique_values = fraud_df[col].unique()
    # numerator = np.sum(np.isin(fraud_df[col], unique_values))
    # denominator = np.sum(np.isin(training_df[col], unique_values))
    # average_ratio = numerator/denominator
    # print(f"real_ratio: {average_ratio}")

    #max and min ratio top 10 and plot
    # unique_values = fraud_df[col].unique()
    # ratio_list = []
    # for value in unique_values:
    #     ratio = fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
    #     ratio_list.append(ratio)
    # ratio_list.sort(reverse=True)
    # if len(ratio_list) > 10:
    #     print(f"max ratio top 10: {ratio_list[:10]}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(np.arange(1, 11), ratio_list[:10])
    #     plt.title(f"Max Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Max Ratio of Fraud Transactions")
    #     plt.show()
    #     print(f"min ratio top 10: {ratio_list[-10:]}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(np.arange(1, 11), ratio_list[-10:])
    #     plt.title(f"Min Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Min Ratio of Fraud Transactions")
    #     plt.show()
    # else:
    #     print(f"max ratio: {ratio_list}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(unique_values, ratio_list)
    #     plt.title(f"Max Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Max Ratio of Fraud Transactions")
    #     plt.show()
    
    # show the max and the min value
    # print(f"max value: {training_df[col].max()}")
    # print(f"min value: {training_df[col].min()}")

    #show the num of ratio 1-0.9 , 0.9-0.8, 0.8-0.7, 0.7-0.6, 0.6-0.5, 0.5-0.4, 0.4-0.3, 0.3-0.2, 0.2-0.1, 0.1-0
    unique_values = fraud_df[col].unique()
    ratio_10_9_num = 0
    ratio_9_8_num = 0
    ratio_8_7_num = 0
    ratio_7_6_num = 0
    ratio_6_5_num = 0
    ratio_5_4_num = 0
    ratio_4_3_num = 0
    ratio_3_2_num = 0
    ratio_2_1_num = 0
    ratio_1_0_num = 0
    for value in unique_values:
        ratio = fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
        if ratio >= 0.9:
            ratio_10_9_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.8:
            ratio_9_8_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.7:
            ratio_8_7_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.6:
            ratio_7_6_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.5:
            ratio_6_5_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.4:
            ratio_5_4_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.3:
            ratio_4_3_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.2:
            ratio_3_2_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.1:
            ratio_2_1_num += fraud_df[col][fraud_df[col] == value].shape[0]
        else:
            ratio_1_0_num += fraud_df[col][fraud_df[col] == value].shape[0]
    # print(f"ratio_10_9_num: {ratio_10_9_num}, ratio: {ratio_10_9_num/ fraud_df.shape[0]}")
    # print(f"ratio_9_8_num: {ratio_9_8_num}, ratio: {ratio_9_8_num/ fraud_df.shape[0]}")
    # print(f"ratio_8_7_num: {ratio_8_7_num}, ratio: {ratio_8_7_num/ fraud_df.shape[0]}")
    # print(f"ratio_7_6_num: {ratio_7_6_num}, ratio: {ratio_7_6_num/ fraud_df.shape[0]}")
    # print(f"ratio_6_5_num: {ratio_6_5_num}, ratio: {ratio_6_5_num/ fraud_df.shape[0]}")
    # print(f"ratio_5_4_num: {ratio_5_4_num}, ratio: {ratio_5_4_num/ fraud_df.shape[0]}")
    # print(f"ratio_4_3_num: {ratio_4_3_num}, ratio: {ratio_4_3_num/ fraud_df.shape[0]}")
    # print(f"ratio_3_2_num: {ratio_3_2_num}, ratio: {ratio_3_2_num/ fraud_df.shape[0]}")
    # print(f"ratio_2_1_num: {ratio_2_1_num}, ratio: {ratio_2_1_num/ fraud_df.shape[0]}")
    # print(f"ratio_1_0_num: {ratio_1_0_num}, ratio: {ratio_1_0_num/ fraud_df.shape[0]}")
    
        
    
    print('\n')


Average ratio:  all unique value in fraud_df and average (amount in fraud_df / amount in training_df) for each feature(column)   
col: label      

---
col: cano       
col: chid       
col: scity      
col: mchno      
col: conam      

---
col: acqic      
col: stocn      
col: csmcu       
col: stscd      
col: flam1      
col: hcefg      
col: mcc        

---
col: csmam      
col: loctm      

---
col: locdt      
col: contp      
col: etymd      
col: ecfg       
col: insfg     
col: iterm      
col: bnsfg     
col: ovrlt   
col: flbmk   
col: flg_3dsmk 

Do feature preprocessing need to do three data part. 
1. training data. : training_df
2. testing data.(pulbic data) : test_df
3. predict data. : testing_private_df

In [9]:
#!!!
temp_training = training_df.copy()
temp_testing = test_df.copy()
temp_prediction = testing_private_df.copy()
temp_y = y.copy()


In [ ]:
training_df = temp_training.copy()
test_df = temp_testing.copy()
testing_private_df = temp_prediction.copy()
y = temp_y.copy()

In [ ]:
training_df =[]
test_df = []
testing_private_df = []
y = temp_y.copy()

Create feature.

In [10]:

#create all columns ratio and ratio_101 then add into prob_ratio_df and prob_ratio_101_df

for col in training_df.columns:
    # print(f"col: {col}")
    #Create a new column : ratio
    prob_ratio_df.loc[:, col + '_ratio'] = 0
    ratio_map = {}
    for value in fraud_df[col].unique():
        ratio = round(fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0],2) 
        ratio_map[value] = ratio*100
    prob_ratio_df.loc[:, col + '_ratio'] = training_df[col].map(ratio_map)
    prob_ratio_df.loc[prob_ratio_df[col + '_ratio'].isna(), col + '_ratio'] = 0
    prob_ratio_df[ col + '_ratio'].astype("category")
    test_prob_ratio_df.loc[:, col + '_ratio'] = test_df[col].map(ratio_map)
    test_prob_ratio_df.loc[test_prob_ratio_df[col + '_ratio'].isna(), col + '_ratio'] = 0
    test_prob_ratio_df[ col + '_ratio'].astype("category")
    private_prob_ratio_df.loc[:, col + '_ratio'] = testing_private_df[col].map(ratio_map)
    private_prob_ratio_df.loc[private_prob_ratio_df[col + '_ratio'].isna(), col + '_ratio'] = 0
    private_prob_ratio_df[ col + '_ratio'].astype("category")
    #Create a new column : ratio_101 
    #ratio_101 is if the row in prob_ratio_df is larger than 90 than 1, 90-10 than 0, smaller than 10 than -1
    #just use prob_ratio_df[col + '_ratio'] to create ratio_101
    prob_ratio_101_df.loc[:, col + '_ratio_101'] = 0
    prob_ratio_101_df.loc[prob_ratio_df[col + '_ratio'] > 90, col + '_ratio_101'] = 1
    prob_ratio_101_df.loc[prob_ratio_df[col + '_ratio'] < 10, col + '_ratio_101'] = -1
    prob_ratio_101_df[ col + '_ratio_101'].astype("category")
    test_prob_ratio_101_df.loc[:, col + '_ratio_101'] = 0
    test_prob_ratio_101_df.loc[test_prob_ratio_df[col + '_ratio'] > 90, col + '_ratio_101'] = 1
    test_prob_ratio_101_df.loc[test_prob_ratio_df[col + '_ratio'] < 10, col + '_ratio_101'] = -1
    test_prob_ratio_101_df[ col + '_ratio_101'].astype("category")
    private_prob_ratio_101_df.loc[:, col + '_ratio_101'] = 0
    private_prob_ratio_101_df.loc[private_prob_ratio_df[col + '_ratio'] > 90, col + '_ratio_101'] = 1
    private_prob_ratio_101_df.loc[private_prob_ratio_df[col + '_ratio'] < 10, col + '_ratio_101'] = -1
    private_prob_ratio_101_df[ col + '_ratio_101'].astype("category")
    

display(prob_ratio_df)
display(prob_ratio_101_df)

,cano,locdt_ratio,loctm_ratio,chid_ratio,cano_ratio,contp_ratio,etymd_ratio,mchno_ratio,acqic_ratio,mcc_ratio,...,flam1_ratio,stocn_ratio,scity_ratio,stscd_ratio,ovrlt_ratio,flbmk_ratio,hcefg_ratio,csmcu_ratio,csmam_ratio,flg_3dsmk_ratio
0,154065,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0,0,0.0,0.0,4.0,0
1,350989,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
2,76572,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
3,167846,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0,0,0.0,0.0,1.0,0
4,407593,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688521,174672,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
8688522,104273,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
8688523,103443,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
8688524,630419,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0


,cano,locdt_ratio_101,loctm_ratio_101,chid_ratio_101,cano_ratio_101,contp_ratio_101,etymd_ratio_101,mchno_ratio_101,acqic_ratio_101,mcc_ratio_101,...,flam1_ratio_101,stocn_ratio_101,scity_ratio_101,stscd_ratio_101,ovrlt_ratio_101,flbmk_ratio_101,hcefg_ratio_101,csmcu_ratio_101,csmam_ratio_101,flg_3dsmk_ratio_101
0,154065,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,350989,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,76572,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,167846,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,407593,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688521,174672,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8688522,104273,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8688523,103443,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8688524,630419,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [11]:
#!!!
#drop the cano in prob_ratio_df and prob_ratio_101_df and test_prob_ratio_df and test_prob_ratio_101_df and private_prob_ratio_df and private_prob_ratio_101_df
prob_ratio_df.drop(['cano'], axis=1, inplace=True)
prob_ratio_101_df.drop(['cano'], axis=1, inplace=True)
test_prob_ratio_df.drop(['cano'], axis=1, inplace=True)
test_prob_ratio_101_df.drop(['cano'], axis=1, inplace=True)
private_prob_ratio_df.drop(['cano'], axis=1, inplace=True)
private_prob_ratio_101_df.drop(['cano'], axis=1, inplace=True)


Deal addtional prob in test_df.

In [ ]:
# addtion_validation_df = pd.concat([training_df, testing_answer_df], axis=1)
display(testing_answer_df)

In [ ]:
# display(training_df[training_df['cano_ratio'].notna()])
# #fill na with 0
# # training_df['cano_ratio'].fillna(0, inplace=True)
# display(training_df[training_df['cano_ratio'].notna()])


# training_df.drop(['chid', 'cano'], axis=1, inplace=True)
# test_df.drop(['chid', 'cano'], axis=1, inplace=True)
# testing_private_df.drop(['chid', 'cano'], axis=1, inplace=True)
# training_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
# test_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
# testing_private_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)

# display(training_df)
# display(test_df)
# display(testing_private_df)
display(prob_ratio_df)
display(prob_ratio_101_df)

# save new feature into temp_training and temp_testing and temp_prediction
# temp_training['cano_ratio'] = training_df['cano_ratio']
# temp_testing['cano_ratio'] = test_df['cano_ratio']
# temp_prediction['cano_ratio'] = testing_private_df['cano_ratio']




In [ ]:
# training_df = pd.concat([prob_ratio_df], axis=1)
# test_df = pd.concat([test_prob_ratio_df], axis=1)
# testing_private_df = pd.concat([private_prob_ratio_df], axis=1)

# training_df = pd.concat([ prob_ratio_101_df], axis=1)
# test_df = pd.concat([ test_prob_ratio_101_df], axis=1)
# testing_private_df = pd.concat([ private_prob_ratio_101_df], axis=1)

# training_df = pd.concat([ prob_ratio_101_df[['chid_ratio_101', 'cano_ratio_101']]], axis=1)
# test_df = pd.concat([ test_prob_ratio_101_df[['chid_ratio_101', 'cano_ratio_101']]], axis=1)
# testing_private_df = pd.concat([ private_prob_ratio_101_df[['chid_ratio_101', 'cano_ratio_101']]], axis=1)

# training_df = pd.concat([prob_ratio_df, prob_ratio_101_df], axis=1)
# test_df = pd.concat([test_prob_ratio_df, test_prob_ratio_101_df], axis=1)
# testing_private_df = pd.concat([private_prob_ratio_df, private_prob_ratio_101_df], axis=1)

Delete features.

In [28]:
#!!!
# #drop columns : csmam, loctm, locdt, contp, etymd, ecfg, insfg, iterm, bnsfg, ovrlt, flbmk, flg_3dsmk
training_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
test_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
testing_private_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)

#drop  columns : csmam_ratio, loctm_ratio, locdt_ratio, contp_ratio, etymd_ratio, ecfg_ratio, insfg_ratio, iterm_ratio, bnsfg_ratio, ovrlt_ratio, flbmk_ratio, flg_3dsmk_ratio
# training_df.drop(['csmam_ratio','loctm_ratio','locdt_ratio','contp_ratio','etymd_ratio','ecfg_ratio','insfg_ratio','iterm_ratio','bnsfg_ratio','ovrlt_ratio','flbmk_ratio','flg_3dsmk_ratio'], axis=1, inplace=True)
# test_df.drop(['csmam_ratio','loctm_ratio','locdt_ratio','contp_ratio','etymd_ratio','ecfg_ratio','insfg_ratio','iterm_ratio','bnsfg_ratio','ovrlt_ratio','flbmk_ratio','flg_3dsmk_ratio'], axis=1, inplace=True)
# testing_private_df.drop(['csmam_ratio','loctm_ratio','locdt_ratio','contp_ratio','etymd_ratio','ecfg_ratio','insfg_ratio','iterm_ratio','bnsfg_ratio','ovrlt_ratio','flbmk_ratio','flg_3dsmk_ratio'], axis=1, inplace=True)

#drop  columns : csmam_ratio_101, loctm_ratio_101, locdt_ratio_101, contp_ratio_101, etymd_ratio_101, ecfg_ratio_101, insfg_ratio_101, iterm_ratio_101, bnsfg_ratio_101, ovrlt_ratio_101, flbmk_ratio_101, flg_3dsmk_ratio_101
# training_df.drop(['csmam_ratio_101','loctm_ratio_101','locdt_ratio_101','contp_ratio_101','etymd_ratio_101','ecfg_ratio_101','insfg_ratio_101','iterm_ratio_101','bnsfg_ratio_101','ovrlt_ratio_101','flbmk_ratio_101','flg_3dsmk_ratio_101'], axis=1, inplace=True)
# test_df.drop(['csmam_ratio_101','loctm_ratio_101','locdt_ratio_101','contp_ratio_101','etymd_ratio_101','ecfg_ratio_101','insfg_ratio_101','iterm_ratio_101','bnsfg_ratio_101','ovrlt_ratio_101','flbmk_ratio_101','flg_3dsmk_ratio_101'], axis=1, inplace=True)
# testing_private_df.drop(['csmam_ratio_101','loctm_ratio_101','locdt_ratio_101','contp_ratio_101','etymd_ratio_101','ecfg_ratio_101','insfg_ratio_101','iterm_ratio_101','bnsfg_ratio_101','ovrlt_ratio_101','flbmk_ratio_101','flg_3dsmk_ratio_101'], axis=1, inplace=True)

C:\Users\lo\AppData\Local\Temp\ipykernel_18800\485317510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_18800\485317510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_18800\485317510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [ ]:
#drop columns : acqic stocn csmcu stscd flam1 hcefg mcc

#deal training data 
training_df.drop(['acqic' ,'stocn' ,'csmcu', 'stscd', 'flam1' ,'hcefg', 'mcc'], axis=1, inplace=True)

#deal testing data
test_df.drop(['acqic' ,'stocn' ,'csmcu', 'stscd', 'flam1' ,'hcefg', 'mcc'], axis=1, inplace=True)


In [ ]:
#deal training data 
# training_df.drop(['cano_ratio' ], axis=1, inplace=True)
# training_df.drop(['cano_ratio_bool' ], axis=1, inplace=True)

#deal testing data
# test_df.drop(['cano_ratio' ], axis=1, inplace=True)
# test_df.drop(['chid_ratio' ], axis=1, inplace=True)
# test_df.drop(['cano_ratio_bool' ], axis=1, inplace=True)




display(training_df.head())
display(test_df.head())





Imbalence part.

In [ ]:
# original smote
# smote = SMOTE(sampling_strategy='auto', random_state=42)

# borderline smote
# smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42, kind='borderline-1')
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.2)

# svmsmote
# smote = SVMSMOTE(sampling_strategy=0.2, random_state=42)

# adasyn
# smote = ADASYN(sampling_strategy=0.2, random_state=42)

# smoteenn
# smote = SMOTEENN(sampling_strategy='auto', random_state=42)

# smotetomek
# smote = SMOTETomek(sampling_strategy=0.2, random_state=42)

smote.fit(training_df, y)
training_df, y = smote.fit_resample(training_df, y)

# fraud_df = training_df[y == 1]
# non_fraud_df = training_df[y == 0]
# print(fraud_df.shape)
# print(non_fraud_df.shape)

# #enn 
# enn = EditedNearestNeighbours()
# training_df, y =  enn.fit_resample(training_df, y)



# fraud_df = training_df[y == 1]
# non_fraud_df = training_df[y == 0]
# print(fraud_df.shape)
# print(non_fraud_df.shape)


split data.

In [32]:
#!!!
# y = training_df['label']
# X = training_df.drop(['label','txkey'], axis=1)
# training_df = training_df.drop(['cano_ratio'])
display(training_df.shape)
X = training_df
#X randomly the rows
# X = X.sample(n=X.shape[0], random_state=42, replace=False)
# display(X)
# display(y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X
y_train = y
# X_test = test_df
# y_test = testing_answer_df['fraud_ind']



(8688526, 12)

## training & testing.
### LightGBM

In [ ]:



# df_sample = df.sample(n=300000, random_state=42)
# train_df, val_df = train_test_split(df_sample, test_size=0.2, random_state=42)

# # Define the features and target variable
# #featues 前面preprocess先做好這邊在註解掉
# features = [ 'locdt','loctm','contp','etymd','mcc','conam','ecfg','insfg','iterm','bnsfg','flam1','stocn','scity','ovrlt','flbmk','hcefg','csmcu','csmam','flg_3dsmk'] 
# target = 'label'

# # Split the training and validation datasets into features and target variable
# X_train = train_df[features]
# y_train = train_df[target]
# X_val = val_df[features]
# y_val = val_df[target]

# # Train the model using LightGBM

# # params = {
# #     'objective': 'binary',
# #     'metric': 'binary_logloss',
# #     'boosting_type': 'dart',
# #     'num_leaves': 50,  
# #     'learning_rate': 0.01,  
# #     'feature_fraction': 0.8  
# # }
# params = {'objective': 'binary',
#           'boosting_type': 'gbdt',
#           'metric': 'auc',
#           'learning_rate': 0.007,
#           'num_leaves': 2**8,
#           'max_depth': -1,
#           'tree_learner':'serial',
#           'colsample_bytree': 0.5,
#           'subsample_freq':1,
#           'subsample':0.7,
#           'n_estimators':10000,
#           'min_data_in_leaf': 106,
#           'max_bin':255,
#           'verbosity': -1,
#           'early_stopping_rounds':100,
#          }


# train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)

# model = lgb.train(params, train_data, num_boost_round=10000, valid_sets=[train_data, val_data])
# model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data, val_data], early_stopping_rounds=50)

categories = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic',
              'mcc',  'ecfg', 'insfg',  'bnsfg', 'stocn', 'scity',
              'stscd', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']

params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 60,
    'max_depth': 6,
    'learning_rate': 0.005,
    'n_estimators': 3000,
    'boosting_type': 'gbdt',
}   

train_data = lgb.Dataset(X_train, label=y_train,
                         categorical_feature=categories)
test_data = lgb.Dataset(X_test, label=y_test,
                        reference=train_data)

bst = lgb.train(params, train_data,num_boost_round = 100,  valid_sets=[test_data],categorical_feature=categories)

validation.

In [ ]:
y_pred_prob = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred2 = [1 if x >= 0.5 else 0 for x in y_pred_prob]  # Assuming binary classification, adjust as needed
# y_pred = model.predict(X_val)
# print(y_pred)
# # Get the true labels from the validation set
# y_true = val_data.label

# # Calculate F1 score
# f1 = f1_score(y_val, y_pred)
# print(f'F1 Score on Validation Set: {f1}')

# y_pred = pd.Series(bst.predict(X_test, num_iteration=bst.best_iteration)>0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred2,digits=5))

### XGboost

In [33]:
#!!!
from xgboost import XGBClassifier
xgboostModel = XGBClassifier( n_estimators=12000, learning_rate= 0.005,enable_categorical=True,
                             objective = "binary:hinge",eval_metric="auc",max_depth=12, device="cuda")#grow_policy="lossguide"
xgboostModel.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=12000, n_jobs=None,
              num_parallel_tree=None, objective='binary:hinge', ...)

easyesemble.

In [ ]:

xgboostModel = XGBClassifier( n_estimators=12000, learning_rate= 0.005,enable_categorical=True,
                             objective = "binary:hinge",eval_metric="auc",max_depth=12, device="cuda")
easy_model = EasyEnsembleClassifier(base_estimator=xgboostModel, n_estimators=10, sampling_strategy='auto', replacement=False, random_state=42, n_jobs=1)
easy_model.fit(X_train, y_train)



double train.

In [ ]:
#use xgboost to predict the training data then use the result and prob_ratio_df and prob_ratio_101_df to train the model by easy_model
# traning_pred = xgboostModel.predict(X_train)
# traning_pred = y_pred
# create new df combine y_pred and prob_ratio_df and prob_ratio_101_df
new_df = pd.DataFrame(traning_pred, columns=['traning_pred'])
new_df = pd.concat([new_df, prob_ratio_df], axis=1)
new_df = pd.concat([new_df, prob_ratio_101_df], axis=1)
easy_model = EasyEnsembleClassifier(base_estimator=xgboostModel, n_estimators=10, sampling_strategy='auto', replacement=False, random_state=42, n_jobs=1)
easy_model.fit(new_df, y_train)


In [ ]:
y_pred = xgboostModel.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred,digits=5))

## predict

Brute

In [35]:
#!!!
# create a dataframe that shape the same as y_pred
# brute_pred = pd.DataFrame(test_df['cano'].rename('y_pred'))
# brute_pred['y_pred'] = 0
y_pred = xgboostModel.predict(test_df)
brute_pred = pd.DataFrame(y_pred, columns=['y_pred'])


# use prob_ratio_df for each column to create a new set of data, if the value is equal to 1 then put it into the new set of data
# then use the new set of data to predict the result
for col in prob_ratio_df.columns:
    col_name = col.replace('_ratio', '')  # Remove '_ratio' from column name
    if col_name in test_df.columns:  # Check if col_name exists in test_df
        the_100 = prob_ratio_df[col][prob_ratio_df[col] == 100].index
        # switch index into value in the_100
        the_100 = training_df.loc[the_100, col_name]
        # if the_100 is not empty, then check if test_df[col] has the value in the_100 and the index is the same as the_100 and set the value to 1
        if the_100.empty == False:
            the_100 = test_df[col_name][test_df[col_name].isin(the_100)].index
            brute_pred.loc[the_100, 'y_pred'] = 1

# change brute_pred into list
brute_pred = brute_pred['y_pred'].tolist()
# display(brute_pred)
#compare y_pred and brute_pred and save it into compare_pred
# if y_pred is 1 and brute_pred is 0 then change compare_pred to 1
# if y_pred is 0 and brute_pred is 1 then change compare_pred to 1
# if y_pred is 1 and brute_pred is 1 then change compare_pred to 1
# if y_pred is 0 and brute_pred is 0 then change compare_pred to 0
# compare_pred = []
# for i in range(len(y_pred)):
#     if y_pred[i] == 1 and brute_pred[i] == 0:
#         compare_pred.append(0)
#     elif y_pred[i] == 0 and brute_pred[i] == 1:
#         compare_pred.append(1)
#     elif y_pred[i] == 1 and brute_pred[i] == 1:
#         compare_pred.append(1)
#     elif y_pred[i] == 0 and brute_pred[i] == 0:
#         compare_pred.append(0)
#     else:
#         print("error")


Use pubic data to validation and test the f1-score.

In [13]:
#!!!
label_encoder_txkey = LabelEncoder()
test_allData = pd.concat([testing_df ,testing_answer_df])
testing_num = testing_df.shape[0]
test_allData['txkey'] = label_encoder_txkey.fit_transform(test_allData['txkey'])
the_testing_df = test_allData[:testing_num]
the_testing_answer_df = test_allData[testing_num:]
# Use map() function to find answer in testing_answer_df using txkey label
ground_truth = []
ground_truth = the_testing_df['txkey'].map(the_testing_answer_df.set_index('txkey')['label'])
ground_truth = ground_truth.tolist()
ground_truth = np.array(ground_truth)



In [37]:
#!!!
#predict test_df than use testing_df's txkey to get the answer in testing_answer_df
#use xgboostModel to training
# display(test_df.shape)
# display(X_train.shape)
# display(ground_truth.nonzero()[0].shape[0])

# switch test_df's column : cano_ratio_bool and cano_ratio 
# temp = test_df['cano_ratio_bool']
# test_df['cano_ratio_bool'] = test_df['cano_ratio']
# test_df['cano_ratio'] = temp
#also swtcich catergory name


# y_pred = xgboostModel.predict(test_df)
# y_pred = easy_model.predict(test_df)

# display(test_df.head())
#use bst to training
# y_pred_prob = bst.predict(test_df, num_iteration=bst.best_iteration)
# y_pred = [1 if x >= 0.5 else 0 for x in y_pred_prob]  



#calculate f1 score
# f1 = f1_score(ground_truth, y_pred)
# f1 = f1_score(ground_truth, compare_pred)
f1 = f1_score(ground_truth, brute_pred)
print(f'F1 Score on Public dataset: {f1}')




F1 Score on Public dataset: 0.6470436354247353


In [ ]:
#use xgboost to predict the training data then use the result and prob_ratio_df and prob_ratio_101_df to train the model by easy_model
# traning_pred = xgboostModel.predict(training_df)
traning_pred = easy_model.predict(training_df)
# create new df combine y_pred and prob_ratio_df and prob_ratio_101_df

# xgboostModel.fit(new_df, y)


In [ ]:
# new_df = pd.DataFrame(traning_pred, columns=['traning_pred'])
# a = prob_ratio_df[['chid_ratio', 'cano_ratio']]
# b = prob_ratio_101_df[['chid_ratio_101', 'cano_ratio_101']]
# new_df = pd.concat([new_df, a], axis=1)
# new_df = pd.concat([new_df, b], axis=1)
# display(new_df.shape)
# display(y.shape)
# xgboostModel.fit(new_df, y)
traning_pred = easy_model.predict(test_df)
new_test_df = pd.DataFrame(traning_pred, columns=['traning_pred'])
new_test_df = pd.concat([new_test_df, test_prob_ratio_df[['chid_ratio', 'cano_ratio']]], axis=1)
new_test_df = pd.concat([new_test_df, test_prob_ratio_101_df[['chid_ratio_101', 'cano_ratio_101']]], axis=1)
y_pred = xgboostModel.predict(new_test_df)
f1 = f1_score(ground_truth, y_pred)
print(f'F1 Score on Public dataset: {f1}')

In [ ]:
display(new_df)
display(new_test_df)

Predict part.

In [40]:
#!!!
y_pred = xgboostModel.predict(testing_private_df)

private_brute_pred = pd.DataFrame(y_pred, columns=['y_pred'])
# use prob_ratio_df for each column to create a new set of data, if the value is equal to 1 then put it into the new set of data
# then use the new set of data to predict the result
for col in prob_ratio_df.columns:
    col_name = col.replace('_ratio', '')  # Remove '_ratio' from column name
    if col_name in testing_private_df.columns:  # Check if col_name exists in test_df
        the_100 = prob_ratio_df[col][prob_ratio_df[col] == 100].index
        # switch index into value in the_100
        the_100 = training_df.loc[the_100, col_name]
        # if the_100 is not empty, then check if test_df[col] has the value in the_100 and the index is the same as the_100 and set the value to 1
        if the_100.empty == False:
            the_100 = testing_private_df[col_name][testing_private_df[col_name].isin(the_100)].index
            private_brute_pred.loc[the_100, 'y_pred'] = 1

# change brute_pred into list
private_brute_pred = private_brute_pred['y_pred'].tolist()

In [ ]:
#use xgboostModel to training
y_pred = xgboostModel.predict(testing_private_df)

# #use bst to training
# predict_data = testing_private_df
# y_pred = pd.Series(bst.predict(predict_data, num_iteration=bst.best_iteration)>0.5).astype(int)


In [43]:
# test_df = pd.read_csv(test_file_path)
# pridict['pred'] = y_pred 
pridict['pred'] = private_brute_pred 

# display(y_pred )
# print(np.nonzero(y_pred))
# print(y_pred[1771])
# display(pridict.iloc[1900])
# display(pridict)
a = pd.DataFrame(testing_df['txkey'] ) # go up to before drop txkey
a['pred'] = ground_truth.astype(int)
output = pd.concat([a, pridict], axis=0)
display(output)

,txkey,pred
0,a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6c...,0
1,16c4880500059e01553789be11bbb50753b7acaae7b95b...,0
2,623c56be3bee87724e3d119c271d9ed098eeda84233183...,0
3,250da12187059cf6e3a3066656a2919d08ceb8207efd55...,0
4,4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843...,0
...,...,...
754134,47cd55bd7492e01e197b61799d072e6ca9e1c118bc76e9...,0
754135,26cef9cc35fb73bedd08a711ebb6d612e4af7dd7b62e7c...,0
754136,f12f861ef6a8517ecddae262c1cc895f618e7e8c19eb9a...,0
754137,638b1a6a6f4389b7af60a6cb0ca908c82d0bdffc6924b0...,0


## output

In [44]:
output.to_csv('..\\output.csv', index=False)
# output.to_csv('output.csv', index=False)
